In [2]:
import pandas as pd
import numpy as np
import math

import statsmodels.api as sm
from statsmodels.sandbox.regression.gmm import IV2SLS

### Part 1) - Part 2)

In [3]:
sigma = .1
var = [[1, sigma ], [sigma , 1]]
mean = [0., 0.]
N = int(1e6)

ey, ex = np.random.multivariate_normal(mean, var, size = N).transpose()

In [4]:
pz = .3

z = np.random.binomial(1,pz,N).astype(np.float)

In [5]:
beta_0x = -.5
beta_1x = 2

x = (beta_0x + beta_1x*z + ex > 0).astype(np.float) 

In [6]:
beta_0y = 1
beta_1y = 1

y = beta_0y + beta_1y*x + ey

### Part 5

$E(y(1) -y(0)) = E(\beta_{0y} + \beta_{1jy} + \epsilon_{yj} |x=1 ) - E(\beta_{0y} + \epsilon_{yj} | x_j=0 ) = E(\beta_{1jy} |x_j =1) = \beta_{1y} + E(\epsilon_{yj}|x_j = 1) -E(\epsilon_{yj}|x_j=0) $

### Part 6

* What happens to the OLS estimator applied to this data?
It does not recover the average treatment effect of $X$ on $Y$
* Is that expected or unexpected?

In [40]:
y1 = (y*x).sum()/x.sum()
y0 =  (y*(1-x)).sum()/(1-x).sum()

y1 - y0

model = sm.OLS(y, sm.add_constant(x))
model = model.fit()
print model.summary()

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.238
Model:                            OLS   Adj. R-squared:                  0.238
Method:                 Least Squares   F-statistic:                 3.115e+05
Date:                Tue, 13 Nov 2018   Prob (F-statistic):               0.00
Time:                        16:51:01   Log-Likelihood:            -1.4187e+06
No. Observations:             1000000   AIC:                         2.837e+06
Df Residuals:                  999998   BIC:                         2.837e+06
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.9415      0.001    668.668      0.0

### Part 7

Using the IV estimator is much closer

In [41]:
y1 = (y*z).sum()/z.sum()
y0 =  (y*(1-z)).sum()/(1-z).sum()

x1 = (x*z).sum()/z.sum()
x0 =  (x*(1-z)).sum()/(1-z).sum()

print (y1 - y0)/(x1 - x0)


model =  IV2SLS(y, sm.add_constant(x), sm.add_constant(z))
model = model.fit()
print model.summary()

1.0006466148033146
                          IV2SLS Regression Results                           
Dep. Variable:                      y   R-squared:                       0.235
Model:                         IV2SLS   Adj. R-squared:                  0.235
Method:                     Two Stage   F-statistic:                 8.191e+04
                        Least Squares   Prob (F-statistic):               0.00
Date:                Tue, 13 Nov 2018                                         
Time:                        16:52:17                                         
No. Observations:             1000000                                         
Df Residuals:                  999998                                         
Df Model:                           1                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.9987      0.002 

### Part 8 


* If $x_i = 1$, and $z_i = 1$ (compliers) then $\epsilon_{xj} > -\beta_{0x} - \beta_{1x}$
* If $x_i = 0$, and $z_i = 1$ (never takers) then $\epsilon_{xj} \leq -\beta_{0x} - \beta_{1x}$
* If $x_i = 1$, and $z_i = 0$ (defiers) then $\epsilon_{xj} > -\beta_{0x} $
* If $x_i = 0$, and $z_i = 0$ (never takers) then $\epsilon_{xj} \leq -\beta_{0x} $


### Part 9 

Above we can see that defiers have $\epsilon_{xj} \leq -\beta_{0x} + \beta_{1x}$
When  $min(\epsilon_{xj}) > -\beta_{0x} + \beta_{1x}$, we have no defiers

### Part 10

In [9]:
c = (x*z).sum()/N 
print 'compliers: %s'%c

d = ((1-x)*z).sum()/N
print 'never takers: %s'%d

a =   (x*(1-z)).sum()/N
print 'defiers: %s'%a

n= ((1-x)*(1-z)).sum()/N
print 'never takers: %s'%n


compliers: 0.279964
never takers: 0.019956
defiers: 0.215909
never takers: 0.484171


### Part 11

In [10]:
c = ( ex > - beta_0x -  beta_1x ).astype(np.float) 

x1c = x*c #x is 1 and a complier
x0c = (1-x)*c #x is 0 and a complier


y1c = (y*x1c).sum()/x1c.sum()
y0c =  (y*x0c).sum()/x0c.sum()

print y1c - y0c

1.0940596057485483


### Part 12 

The average treatment effect is the LATE. The IV estimate is not the LATE

### Part 13

Since $\beta_{1jy} =1$ for all $j$, the treatment effect and the local average treatment effect should be the same. Everyone has the same treatment effect essentially. As a result, it is unclear how to interpret the IV estimate

### Part 14

In [18]:
beta_1jy = np.random.normal(1, 1, size = N)

yj = beta_0y + beta_1jy*x + ey

### Part 15


$E(y(1) -y(0)) = E(\beta_{0y} + \beta_{1jy} + \epsilon_{yj} |x=1 ) - E(\beta_{0y} + \epsilon_{yj} | x_j=0 ) =  E(\beta_{1jy} + \epsilon_{yj}|x_j = 1) -E(\epsilon_{yj}|x_j=0) $

### Part 16

Yes it does, this is expected because the individual coefficient was independent from the data

In [43]:
yj1 = (yj*x).sum()/x.sum()
yj0 =  (yj*(1-x)).sum()/(1-x).sum()

print yj1 - yj0

model = sm.OLS(yj, sm.add_constant(x))
model = model.fit()
print model.summary()

1.1188063193204991
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.173
Model:                            OLS   Adj. R-squared:                  0.173
Method:                 Least Squares   F-statistic:                 2.093e+05
Date:                Tue, 13 Nov 2018   Prob (F-statistic):               0.00
Time:                        16:56:05   Log-Likelihood:            -1.6199e+06
No. Observations:             1000000   AIC:                         3.240e+06
Df Residuals:                  999998   BIC:                         3.240e+06
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.9415      0.002 

### Part 17

In [44]:
yj1 = (yj*z).sum()/z.sum()
yj0 =  (yj*(1-z)).sum()/(1-z).sum()

x1 = (x*z).sum()/z.sum()
x0 =  (x*(1-z)).sum()/(1-z).sum()

print (yj1 - yj0)/(x1 - x0)

model =  IV2SLS(yj, sm.add_constant(x), sm.add_constant(z))
model = model.fit()
print model.summary()

1.0057087091703603
                          IV2SLS Regression Results                           
Dep. Variable:                      y   R-squared:                       0.171
Model:                         IV2SLS   Adj. R-squared:                  0.171
Method:                     Two Stage   F-statistic:                 5.539e+04
                        Least Squares   Prob (F-statistic):               0.00
Date:                Tue, 13 Nov 2018                                         
Time:                        16:56:49                                         
No. Observations:             1000000                                         
Df Residuals:                  999998                                         
Df Model:                           1                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.9976      0.002 

### Part 18

They do not. The coefficient is random, but it does not depend on $\epsilon_x$ or $\epsilon_y$

### Part 19

In [21]:
yj1c = (yj*x1c).sum()/x1c.sum()
yj0c =  (yj*x0c).sum()/x0c.sum()

print yj1c - yj0c

1.096921552257902


### Part 20
* 19 and 15 are roughly the same
* The IV estimate is not the local average treatment effect

### Part 21
The local average treatment effect is the average treatment effect
### Part 22

In [35]:
beta_1iy = np.random.normal(1 + ex, 1, size = N)

yi = beta_0y + beta_1iy*x + ey

### Part 23
The average treatment effect increases

### Part 24
Below we calculate the OLS estimator based on the numerical simulation

In [48]:
yi1 = (yi*x).sum()/x.sum()
yi0 =  (yi*(1-x)).sum()/(1-x).sum()

print yi1 - yi0

model = sm.OLS(yi, sm.add_constant(x))
model = model.fit()
print model.summary()

1.6918891102800409
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.266
Model:                            OLS   Adj. R-squared:                  0.266
Method:                 Least Squares   F-statistic:                 3.631e+05
Date:                Tue, 13 Nov 2018   Prob (F-statistic):               0.00
Time:                        16:58:13   Log-Likelihood:            -1.7581e+06
No. Observations:             1000000   AIC:                         3.516e+06
Df Residuals:                  999998   BIC:                         3.516e+06
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.9415      0.002 

### Part 25

In [46]:
yi1 = (yi*z).sum()/z.sum()
yi0 =  (yi*(1-z)).sum()/(1-z).sum()

x1 = (x*z).sum()/z.sum()
x0 =  (x*(1-z)).sum()/(1-z).sum()

print (yi1 - yi0)/(x1 - x0)

model =  IV2SLS(yi, sm.add_constant(x), sm.add_constant(z))
model = model.fit()
print model.summary()

0.6419121526807274
                          IV2SLS Regression Results                           
Dep. Variable:                      y   R-squared:                       0.164
Model:                         IV2SLS   Adj. R-squared:                  0.164
Method:                     Two Stage   F-statistic:                 1.505e+04
                        Least Squares   Prob (F-statistic):               0.00
Date:                Tue, 13 Nov 2018                                         
Time:                        16:57:53                                         
No. Observations:             1000000                                         
Df Residuals:                  999998                                         
Df Model:                           1                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.4621      0.003 

### Part 26

They increase

### Part 27

In [38]:
yi1c = (yi*x1c).sum()/x1c.sum()
yi0c =  (yi*x0c).sum()/x0c.sum()

print yi1c - yi0c

1.6700043432174438


### Part 28

It is the same